In [0]:
def setpath():
    from pyspark.sql.functions import current_date, input_file_name
    from datetime import date
    source_path = "s3://databricksbijubucketnew/raw/"
    archive_base = "s3://databricksbijubucketnew/archive/"
    checkpoint = "s3://databricksbijubucketnew/checkpoints/"

In [0]:
setpath()

In [0]:
    source_path = "s3://databricksbijubucketnew/raw/"
    archive_base = "s3://databricksbijubucketnew/archive/"
    checkpoint = "s3://databricksbijubucketnew/checkpoints/"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType,DoubleType,DateType

acebook_schema = StructType([
    StructField("currency", StringType(), True),
    StructField("account", StringType(), True),
    StructField("account_name", StringType(), True),
    StructField("campaign_id", StringType(), True),
    StructField("campaign_name", StringType(), True),
    StructField("adset_id", StringType(), True),
    StructField("adset_name", StringType(), True),
    StructField("ad_id", StringType(), True),
    StructField("ad_name", StringType(), True),
    StructField("ad_creation_time", StringType(), True),  # Use TimestampType if you want to parse
    StructField("ad_delivery_status", StringType(), True),
    StructField("objective", StringType(), True),
    StructField("clicks", IntegerType(), True),
    StructField("impressions", IntegerType(), True),
    StructField("reach", IntegerType(), True),
    StructField("frequency", DoubleType(), True),
    StructField("ctr", DoubleType(), True),
    StructField("cpc", DoubleType(), True),
    StructField("cpm", DoubleType(), True),
    StructField("spend", DoubleType(), True),
    StructField("video_plays", IntegerType(), True),
    StructField("page_engagement", IntegerType(), True),
    StructField("conversions", IntegerType(), True),
    StructField("conversion_value", DoubleType(), True),
    StructField("purchase_roas", DoubleType(), True),
    StructField("actions", IntegerType(), True),
    StructField("age", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("region", StringType(), True),
    StructField("date", StringType(), True)  # Use DateType if needed
])




In [0]:
spark.sql("DROP TABLE IF EXISTS bijucatalog.bijubronzeschema.bronze_facebook")


In [0]:
%python
# Read CSV using Auto Loader
from datetime import date
df_bronze = (spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "csv")
  .option("header", "true")  
  .option("multiLine", "true")
  .option("cloudFiles.schemaLocation", checkpoint)
  .option("cloudFiles.maxFilesPerTrigger", 1)
  .option("cloudFiles.validateOptions", "false")
  #.schema(facebookschema)
  .option("inferSchema", True)
  .load(source_path))

# Write to Delta
(df_bronze.writeStream
 .format("delta")
 .outputMode("append")
 .option("checkpointLocation", f"{checkpoint}/data")
 .trigger(availableNow=True)
 .toTable("bijucatalog.bijubronzeschema.bronze_facebook"))

In [0]:
# dbutils.fs.mv("s3://databricksbijubucketnew/raw/",  f"{archive_base}/{date.today().isoformat()}",recurse=True)

In [0]:
%sql
select * from bijucatalog.bijubronzeschema.bronze_facebook

In [0]:
%sql
select count(* ) from  bijucatalog.bijubronzeschema.bronze_facebook

In [0]:
df = spark.read.option("header", True).csv("s3://databricksbijubucketnew/raw/facebook_ads_2025-06-26.csv")
df.show()

In [0]:
df = spark.read.option("header", True).option("inferSchema", True).csv("s3://databricksbijubucketnew/raw/facebook_ads_2025-06-26.csv")
df.printSchema()

In [0]:
# Example to test permissions in a Databricks notebook
# Replace with your actual paths and a dummy file
dbutils.fs.put("s3://databricksbijubucketnew/raw/facebook_data_2025-07-01.csv", "test content", True)
try:
    dbutils.fs.mv("s3://databricksbijubucketnew/raw/facebook_data_2025-07-01.csv", "s3://databricksbijubucketnew/archive/facebook_data_2025-07-01.csv")
    print("Manual move successful! Permissions are likely correct.")
except Exception as e:
    print(f"Manual move failed: {e}. Check your storage permissions!")